In [55]:
import numpy as np
import pandas as pd
import cx_Oracle

In [2]:
# 시계열 정보
time_table1 =[
    ['2012-08-01', '2012-12-31'],
    ['2013-01-01', '2013-12-31'],
    ['2014-01-01', '2014-12-31'],
    ['2015-01-01', '2015-12-31'],
    ['2016-01-01', '2016-12-31'],
    ['2017-01-01', '2017-12-31'],
    ['2018-01-01', '2018-12-31'],
    ['2019-01-01', '2019-12-31'],
    ['2020-01-01', '2020-07-31']]

time_table2 = [
    '2012', '2013', '2014', '2015', '2016',
    '2017', '2018', '2019', '2020']

In [3]:
# 지역 정보
region_table1 = [
    '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시',
    '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시',
    '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']

In [4]:
def get_data_from_db(query):
    conn = cx_Oracle.connect('hoseo', 'hoseo', 'localhost:1521/xe')
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [17]:
# 년도별 지역별 전체 매매건수
def make_region_deal_count():
    for year in time_table2:

        query = f"""
            SELECT adp.region, COUNT(*) AS y{year} 
            FROM apt_deal_price adp 
            WHERE TO_CHAR(adp.contract_date, 'YYYY') = '{year}' 
            GROUP BY adp.region 
            ORDER BY adp.region ASC
            """

        queD = get_data_from_db(query)

        if year == '2012':
            dataD = queD.copy()
        else:
            dataD = dataD.merge(queD)

    dataD.info()
    dataD.to_csv('sub-data-files/region_deal_count.csv', index=False, encoding='utf-8-sig')
    

In [6]:
# make_region_deal_count()

In [7]:
testD = pd.read_csv('sub-data-files/region_deal_count.csv')
testD

,REGION,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,강원도,6630,17830,19607,22038,20351,16608,12825,13914,12550
1,경기도,41582,147064,172370,200666,175042,154053,151747,141529,150660
2,경상남도,12818,39806,47725,48027,37483,30780,25463,33552,24848
3,경상북도,11594,29417,27867,27196,20114,20705,18861,22647,18548
4,광주광역시,8387,22310,25424,23859,21420,22764,25384,20135,13465
5,대구광역시,16495,39240,34041,32299,19047,32067,32582,30142,21410
6,대전광역시,6432,20935,20923,21186,22785,20283,19602,27390,17102
7,부산광역시,13255,42777,52758,62263,56409,37938,25857,36080,30418
8,서울특별시,18633,68136,85537,120023,110183,105063,81389,74917,50428
9,세종특별자치시,546,1130,1204,2033,3520,4669,3430,5802,5722


In [8]:
# 국민임대주택 규모
# 전용면적 50m²(15평) 미만, 무주택 세대주 월평균 소득 50% 이하
# 전용면적 50m²(15평)이상∼60m²(18평) 이하, 무주택 세대주 월평균 소득 70% 이하

# 국민주택 규모
# 전용면적 85m²(25.7평) 이하
# 전용면적 100m²(30.2평) 이하, 수도권을 제외한 도시지역이 아닌 읍·면 지역

# 전용면적 100m²(30.2평)초과~135m²(40.8평)이하
# 전용면적 135m²(40.8평)초과

# ~ 50m²
# 50m² ~ 60m²
# 60m² ~ 85m²
# 85m² ~ 100m²
# 100m² ~ 135m²
# 135m² ~

In [9]:
size_conditions = [
    ('under_50', 'adp.apt_size < 50'),
    ('50_60', 'adp.apt_size >= 50 AND adp.apt_size <= 60'),
    ('60_85', 'adp.apt_size > 60 AND adp.apt_size <= 85'),
    ('85_100', 'adp.apt_size > 85 AND adp.apt_size <= 100'),
    ('100_135', 'adp.apt_size > 100 AND adp.apt_size <= 135'),
    ('135_over', 'adp.apt_size > 135')]

col_apt_size_data = [
    'region', 'apt_size', 'y2012', 'y2013', 'y2014',
    'y2015', 'y2016', 'y2017', 'y2018', 'y2019', 'y2020']
    

In [13]:
# 지역별 사이즈별 매매건수 데이터 수집
def make_region_size_count(year = 2012):
    for cond in size_conditions:

        query = f"""
            SELECT adp.region AS region, COUNT(adp.apt_size) AS "{cond[0]}"
            FROM apt_deal_price adp
            WHERE to_char(adp.contract_date, 'YYYY') = '{year}' AND {cond[1]}
            GROUP BY adp.region
            ORDER BY adp.region ASC
            """

        queD = get_data_from_db(query)

        if cond[0] == 'under_50':
            dataD = queD.copy()
        else:
            dataD = dataD.merge(queD, how='left')
        
    dataD.fillna(0, inplace=True)
    dataD = dataD.astype( \
        {'under_50':'int64', '50_60':'int64', '60_85':'int64',
        '85_100':'int64', '100_135':'int64', '135_over':'int64'})
    
    # dataD.info()

    dataD.set_index('REGION', inplace=True)
    data_list = dataD.values.reshape(len(region_table1) * len(size_conditions), )

    return data_list


In [14]:
# 년도별 지역별 사이즈별 매매건수 데이터 만들기
def complete_region_size_count():

    # 최종 데이터 테이블 형태 만들기
    row_count = len(region_table1) * len(size_conditions)
    col_count = len(col_apt_size_data)
    allD = pd.DataFrame(np.zeros((row_count, col_count), dtype='int64'), columns=col_apt_size_data)

    size_list = [ x[0] for x in size_conditions ]
    allD['apt_size'] = size_list * len(region_table1)

    region_list = []
    for r in region_table1:
        region_list.extend([r] * len(size_conditions))
    allD['region'] = region_list

    # 년도별 데이터 수집
    for year in time_table2:
        data_list = make_region_size_count(year)
        allD['y' + year] = data_list
        
    allD.info()
    allD.to_csv('sub-data-files/region_size_deal_count.csv', index=False, encoding='utf-8-sig')

In [15]:
# complete_region_size_count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    102 non-null    object
 1   apt_size  102 non-null    object
 2   y2012     102 non-null    int64 
 3   y2013     102 non-null    int64 
 4   y2014     102 non-null    int64 
 5   y2015     102 non-null    int64 
 6   y2016     102 non-null    int64 
 7   y2017     102 non-null    int64 
 8   y2018     102 non-null    int64 
 9   y2019     102 non-null    int64 
 10  y2020     102 non-null    int64 
dtypes: int64(9), object(2)
memory usage: 8.9+ KB


In [16]:
testD = pd.read_csv('sub-data-files/region_size_deal_count.csv')
testD.head(12)

,region,apt_size,y2012,y2013,y2014,y2015,y2016,y2017,y2018,y2019,y2020
0,강원도,under_50,1745,5230,5375,5129,4973,4335,2953,2599,2222
1,강원도,50_60,2164,5709,6467,6971,6165,5092,4104,4500,4139
2,강원도,60_85,2063,5477,6316,7868,7326,5814,4712,5820,5144
3,강원도,85_100,75,194,218,278,262,198,135,154,160
4,강원도,100_135,432,894,992,1443,1282,937,742,660,704
5,강원도,135_over,151,326,239,349,343,232,179,181,181
6,경기도,under_50,5283,17442,23048,26504,23472,20073,17546,15751,16352
7,경기도,50_60,11330,40630,46031,52233,46651,40635,37919,35299,38755
8,경기도,60_85,17987,63772,73604,86951,75077,65850,67028,64718,69997
9,경기도,85_100,638,2393,3079,3755,3081,2675,2951,3153,3176


In [37]:
def get_yyyymm():
    query = """
        SELECT DISTINCT to_char(adp.contract_date, 'YYYYMM') AS ym
        FROM apt_deal_price adp
        ORDER BY ym ASC
        """

    queD = get_data_from_db(query)
    return list(queD['YM'])
    

In [40]:
# 월별 지역별 전체 매매건수
def make_region_deal_count_month():
    yyyymm = get_yyyymm()

    for ym in yyyymm:

        query = f"""
            SELECT adp.region, COUNT(*) AS ym{ym} 
            FROM apt_deal_price adp 
            WHERE TO_CHAR(adp.contract_date, 'YYYYMM') = '{ym}' 
            GROUP BY adp.region 
            ORDER BY adp.region ASC
            """

        queD = get_data_from_db(query)

        if ym == '201208':
            dataD = queD.copy()
        else:
            dataD = dataD.merge(queD, how='left')

    dataD.info()
    dataD.to_csv('sub-data-files/region_deal_count_month.csv', index=False, encoding='utf-8-sig')

In [43]:
# make_region_deal_count_month()

In [54]:
testD = pd.read_csv('sub-data-files/region_deal_count_month.csv')
testD.head()

,REGION,YM201208,YM201209,YM201210,YM201211,YM201212,YM201301,YM201302,YM201303,YM201304,...,YM201910,YM201911,YM201912,YM202001,YM202002,YM202003,YM202004,YM202005,YM202006,YM202007
0,강원도,933,1191,1495,1513,1498,1023,1227,2383,1660,...,1343,1377,1751,1439,1868,1579,1625,1982,2312,1745
1,경기도,5631,8083,10645,9178,8045,7115,8588,13003,14632,...,18101,20779,20988,20747,31851,16368,12759,16979,34952,17004
2,경상남도,1623,2113,3168,3101,2813,2286,2801,3747,4001,...,4026,4869,3822,3183,3704,2625,2633,3376,5764,3563
3,경상북도,1781,2087,2701,2519,2506,1947,2324,2837,3134,...,2549,2514,2400,2193,2689,1915,2083,3392,3843,2433
4,광주광역시,1204,1483,2012,1956,1732,1413,1799,2206,2051,...,2046,2133,2128,1954,2251,1738,1490,1778,2423,1831


## 아래 내용은 작업중..

In [56]:
# 월별 지역별 사이즈별 매매건수 데이터 수집
def make_region_size_count_month(ym = 201208):
    for cond in size_conditions:

        query = f"""
            SELECT adp.region AS region, COUNT(NVL(adp.apt_size, 0)) AS "{cond[0]}"
            FROM apt_deal_price adp
            WHERE to_char(adp.contract_date, 'YYYYMM') = '{ym}' AND {cond[1]}
            GROUP BY adp.region
            ORDER BY adp.region ASC
            """

        queD = get_data_from_db(query)

        if cond[0] == 'under_50':
            dataD = queD.copy()
        else:
            dataD = dataD.merge(queD, how='left')
    
    # dataD empty 처리

    dataD.fillna(0, inplace=True)
    dataD = dataD.astype( \
        {'under_50':'int64', '50_60':'int64', '60_85':'int64',
        '85_100':'int64', '100_135':'int64', '135_over':'int64'})
    
    # dataD.info()

    dataD.set_index('REGION', inplace=True)
    data_list = dataD.values.reshape(len(region_table1) * len(size_conditions), )

    return data_list

In [57]:
# 월별 지역별 사이즈별 매매건수 데이터 만들기
def complete_region_size_count():
    yyyymm = get_yyyymm()
    col_ym = [ 'ym' + y for y in yyyymm ]
    col_apt_size_month = ['region', 'apt_size']
    col_apt_size_month += col_ym

    # 최종 데이터 테이블 형태 만들기
    row_count = len(region_table1) * len(size_conditions)
    col_count = len(col_apt_size_month)
    allD = pd.DataFrame(np.zeros((row_count, col_count), dtype='int64'), columns=col_apt_size_month)

    size_list = [ x[0] for x in size_conditions ]
    allD['apt_size'] = size_list * len(region_table1)

    region_list = []
    for r in region_table1:
        region_list.extend([r] * len(size_conditions))
    allD['region'] = region_list

    # 년도별 데이터 수집
    for ym in yyyymm:
        data_list = make_region_size_count(ym)
        allD['ym' + ym] = data_list
        
    allD.info()
    allD.to_csv('sub-data-files/region_size_deal_count_month.csv', index=False, encoding='utf-8-sig')